# **Toronto Crime Predictions**

In [2]:
# Dependencies
import requests
import json
import pprint #
import pandas as pd
from sklearn.preprocessing import StandardScaler #
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.cluster import DBSCAN #

---

## **Retrieve Data from API**

In [3]:
# Initialize the base URL and the initial parameters
base_url = 'https://services.arcgis.com/S9th0jAJ7bqgIRjw/ArcGIS/rest/services/Major_Crime_Indicators_Open_Data/FeatureServer/0/query'
params = {
    'where': 'OCC_YEAR<=2024 AND OCC_YEAR>=2021',
    'outFields': '*',
    'resultRecordCount': 2000,
    'resultOffset': 0,
    'f': 'geojson'
}
# Create an empty list to store all the results
crime_json_array = []
# Start the loop to paginate through the results
while True:
    response = requests.get(base_url, params=params).json()
    # Add the current batch of features to the list
    crime_json_array.extend(response['features'])
    # Check if 'exceededTransferLimit' is True, which means there are more records
    if 'properties' in response and response['properties'].get('exceededTransferLimit', False):
        # Increment the offset to get the next batch of records
        params['resultOffset'] += params['resultRecordCount']
    else:
        # Exit the loop if there are no more records to retrieve
        break

In [4]:
# View the raw JSON data
crime_json_array

[{'type': 'Feature',
  'id': 246675,
  'geometry': {'type': 'Point',
   'coordinates': [-79.425761926, 43.6817690130001]},
  'properties': {'OBJECTID': 246675,
   'EVENT_UNIQUE_ID': 'GO-20213605',
   'REPORT_DATE': 1609477200000,
   'OCC_DATE': 1609477200000,
   'REPORT_YEAR': 2021,
   'REPORT_MONTH': 'January',
   'REPORT_DAY': 1,
   'REPORT_DOY': 1,
   'REPORT_DOW': 'Friday    ',
   'REPORT_HOUR': 16,
   'OCC_YEAR': 2021,
   'OCC_MONTH': 'January',
   'OCC_DAY': 1,
   'OCC_DOY': 1,
   'OCC_DOW': 'Friday    ',
   'OCC_HOUR': 16,
   'DIVISION': 'D13',
   'LOCATION_TYPE': 'Parking Lots (Apt., Commercial Or Non-Commercial)',
   'PREMISES_TYPE': 'Outside',
   'UCR_CODE': 2135,
   'UCR_EXT': 210,
   'OFFENCE': 'Theft Of Motor Vehicle',
   'MCI_CATEGORY': 'Auto Theft',
   'HOOD_158': '094',
   'NEIGHBOURHOOD_158': 'Wychwood (94)',
   'HOOD_140': '094',
   'NEIGHBOURHOOD_140': 'Wychwood (94)',
   'LONG_WGS84': -79.42576192637651,
   'LAT_WGS84': 43.68176901263976}},
 {'type': 'Feature',
  'i

In [5]:
def arrange_crime_json_array(crime):
    """ Function for filtering out the API query metdata, and getting a JSON list of just the crime data"""
    crime_case = crime['properties']
    return crime_case

In [6]:
# Using a list comprehension to apply the function for each entry in the intial crime JSON array.
crime_json_list = [crime['properties'] for crime in crime_json_array]
crime_json_list

[{'OBJECTID': 246675,
  'EVENT_UNIQUE_ID': 'GO-20213605',
  'REPORT_DATE': 1609477200000,
  'OCC_DATE': 1609477200000,
  'REPORT_YEAR': 2021,
  'REPORT_MONTH': 'January',
  'REPORT_DAY': 1,
  'REPORT_DOY': 1,
  'REPORT_DOW': 'Friday    ',
  'REPORT_HOUR': 16,
  'OCC_YEAR': 2021,
  'OCC_MONTH': 'January',
  'OCC_DAY': 1,
  'OCC_DOY': 1,
  'OCC_DOW': 'Friday    ',
  'OCC_HOUR': 16,
  'DIVISION': 'D13',
  'LOCATION_TYPE': 'Parking Lots (Apt., Commercial Or Non-Commercial)',
  'PREMISES_TYPE': 'Outside',
  'UCR_CODE': 2135,
  'UCR_EXT': 210,
  'OFFENCE': 'Theft Of Motor Vehicle',
  'MCI_CATEGORY': 'Auto Theft',
  'HOOD_158': '094',
  'NEIGHBOURHOOD_158': 'Wychwood (94)',
  'HOOD_140': '094',
  'NEIGHBOURHOOD_140': 'Wychwood (94)',
  'LONG_WGS84': -79.42576192637651,
  'LAT_WGS84': 43.68176901263976},
 {'OBJECTID': 246676,
  'EVENT_UNIQUE_ID': 'GO-20213400',
  'REPORT_DATE': 1609477200000,
  'OCC_DATE': 1609477200000,
  'REPORT_YEAR': 2021,
  'REPORT_MONTH': 'January',
  'REPORT_DAY': 1,
  

In [7]:
# Convert the JSON array into a Dataframe
crime_data_original = pd.DataFrame(crime_json_list)
crime_data_original

,OBJECTID,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,REPORT_DOW,REPORT_HOUR,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,246675,GO-20213605,1609477200000,1609477200000,2021,January,1,1,Friday,16,...,2135,210,Theft Of Motor Vehicle,Auto Theft,094,Wychwood (94),094,Wychwood (94),-79.425762,43.681769
1,246676,GO-20213400,1609477200000,1609477200000,2021,January,1,1,Friday,16,...,2135,210,Theft Of Motor Vehicle,Auto Theft,NSA,NSA,NSA,NSA,0.000000,0.000000
2,246677,GO-20211123,1609477200000,1609477200000,2021,January,1,1,Friday,7,...,2135,210,Theft Of Motor Vehicle,Auto Theft,031,Yorkdale-Glen Park (31),031,Yorkdale-Glen Park (31),-79.460110,43.721013
3,246678,GO-2021445,1609477200000,1609477200000,2021,January,1,1,Friday,1,...,2135,210,Theft Of Motor Vehicle,Auto Theft,151,Yonge-Doris (151),051,Willowdale East (51),-79.415293,43.778743
4,246679,GO-20213400,1609477200000,1609477200000,2021,January,1,1,Friday,16,...,2135,210,Theft Of Motor Vehicle,Auto Theft,NSA,NSA,NSA,NSA,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147546,396731,GO-20241427047,1719723600000,1719637200000,2024,June,30,182,Sunday,16,...,1430,100,Assault,Assault,071,Cabbagetown-South St.James Town (71),071,Cabbagetown-South St.James Town (71),-79.373043,43.663195
147547,396732,GO-20241427869,1719723600000,1719723600000,2024,June,30,182,Sunday,18,...,2133,200,Theft Over - Shoplifting,Theft Over,027,York University Heights (27),027,York University Heights (27),-79.464942,43.759469
147548,396733,GO-20241423116,1719723600000,1719637200000,2024,June,30,182,Sunday,2,...,1450,120,Discharge Firearm With Intent,Assault,144,Morningside Heights (144),131,Rouge (131),-79.248477,43.837237
147549,396734,GO-20241426669,1719723600000,1718859600000,2024,June,30,182,Sunday,15,...,2132,200,Theft From Motor Vehicle Over,Theft Over,160,Mimico-Queensway (160),017,Mimico (includes Humber Bay Shores) (17),-79.521053,43.616490




---



## **Preprocess the Data**

In [8]:
# Get all the column names in crime_data_original
crime_data_original.columns

Index(['OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE', 'REPORT_YEAR',
       'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW', 'REPORT_HOUR',
       'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR',
       'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE', 'UCR_EXT',
       'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140',
       'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'],
      dtype='object')

In [9]:
# Change the value of 'NSA' in the HOOD_158 column to 0 and then convert the column values to integers
crime_data_original['HOOD_158'] = crime_data_original['HOOD_158'].replace('NSA', 0)
crime_data_original['HOOD_158'] = crime_data_original['HOOD_158'].astype(int)
crime_data_original

,OBJECTID,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,REPORT_DOW,REPORT_HOUR,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,246675,GO-20213605,1609477200000,1609477200000,2021,January,1,1,Friday,16,...,2135,210,Theft Of Motor Vehicle,Auto Theft,94,Wychwood (94),094,Wychwood (94),-79.425762,43.681769
1,246676,GO-20213400,1609477200000,1609477200000,2021,January,1,1,Friday,16,...,2135,210,Theft Of Motor Vehicle,Auto Theft,0,NSA,NSA,NSA,0.000000,0.000000
2,246677,GO-20211123,1609477200000,1609477200000,2021,January,1,1,Friday,7,...,2135,210,Theft Of Motor Vehicle,Auto Theft,31,Yorkdale-Glen Park (31),031,Yorkdale-Glen Park (31),-79.460110,43.721013
3,246678,GO-2021445,1609477200000,1609477200000,2021,January,1,1,Friday,1,...,2135,210,Theft Of Motor Vehicle,Auto Theft,151,Yonge-Doris (151),051,Willowdale East (51),-79.415293,43.778743
4,246679,GO-20213400,1609477200000,1609477200000,2021,January,1,1,Friday,16,...,2135,210,Theft Of Motor Vehicle,Auto Theft,0,NSA,NSA,NSA,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147546,396731,GO-20241427047,1719723600000,1719637200000,2024,June,30,182,Sunday,16,...,1430,100,Assault,Assault,71,Cabbagetown-South St.James Town (71),071,Cabbagetown-South St.James Town (71),-79.373043,43.663195
147547,396732,GO-20241427869,1719723600000,1719723600000,2024,June,30,182,Sunday,18,...,2133,200,Theft Over - Shoplifting,Theft Over,27,York University Heights (27),027,York University Heights (27),-79.464942,43.759469
147548,396733,GO-20241423116,1719723600000,1719637200000,2024,June,30,182,Sunday,2,...,1450,120,Discharge Firearm With Intent,Assault,144,Morningside Heights (144),131,Rouge (131),-79.248477,43.837237
147549,396734,GO-20241426669,1719723600000,1718859600000,2024,June,30,182,Sunday,15,...,2132,200,Theft From Motor Vehicle Over,Theft Over,160,Mimico-Queensway (160),017,Mimico (includes Humber Bay Shores) (17),-79.521053,43.616490


In [10]:
crime_data_original['OCC_MONTH']

0         January
1         January
2         January
3         January
4         January
           ...   
147546       June
147547       June
147548       June
147549       June
147550       June
Name: OCC_MONTH, Length: 147551, dtype: object

In [11]:
# prompt: Convert the crime_data_original column OCC_MONTH's month names to integer form

month_mapping = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

crime_data_original['OCC_MONTH'] = crime_data_original['OCC_MONTH'].map(month_mapping)
crime_data_original['OCC_MONTH']


0         1
1         1
2         1
3         1
4         1
         ..
147546    6
147547    6
147548    6
147549    6
147550    6
Name: OCC_MONTH, Length: 147551, dtype: int64

In [12]:
# Collect only the necessary columns for the analysis
crime_pd = crime_data_original[['EVENT_UNIQUE_ID', 'NEIGHBOURHOOD_158', 'HOOD_158', 'LAT_WGS84', 'LONG_WGS84', 'PREMISES_TYPE', 'OCC_DATE', 'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_HOUR', 'MCI_CATEGORY']]
crime_pd


,EVENT_UNIQUE_ID,NEIGHBOURHOOD_158,HOOD_158,LAT_WGS84,LONG_WGS84,PREMISES_TYPE,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_HOUR,MCI_CATEGORY
0,GO-20213605,Wychwood (94),94,43.681769,-79.425762,Outside,1609477200000,2021,1,1,16,Auto Theft
1,GO-20213400,NSA,0,0.000000,0.000000,Commercial,1609477200000,2021,1,1,4,Auto Theft
2,GO-20211123,Yorkdale-Glen Park (31),31,43.721013,-79.460110,Other,1609477200000,2021,1,1,4,Auto Theft
3,GO-2021445,Yonge-Doris (151),151,43.778743,-79.415293,Other,1609477200000,2021,1,1,1,Auto Theft
4,GO-20213400,NSA,0,0.000000,0.000000,Commercial,1609477200000,2021,1,1,4,Auto Theft
...,...,...,...,...,...,...,...,...,...,...,...,...
147546,GO-20241427047,Cabbagetown-South St.James Town (71),71,43.663195,-79.373043,Apartment,1719637200000,2024,6,29,23,Assault
147547,GO-20241427869,York University Heights (27),27,43.759469,-79.464942,Commercial,1719723600000,2024,6,30,18,Theft Over
147548,GO-20241423116,Morningside Heights (144),144,43.837237,-79.248477,Outside,1719637200000,2024,6,29,21,Assault
147549,GO-20241426669,Mimico-Queensway (160),160,43.616490,-79.521053,Outside,1718859600000,2024,6,20,13,Theft Over


In [13]:
# Encode the MCI_CATEGORY column using get_dummies
crime_category_encoded = pd.get_dummies(crime_pd['MCI_CATEGORY']).astype(int)
crime_category_encoded


,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
147546,1,0,0,0,0
147547,0,0,0,0,1
147548,1,0,0,0,0
147549,0,0,0,0,1


In [14]:
# Add the encoded columns back into the datframe
crime_pd = pd.concat([crime_pd, crime_category_encoded], axis=1)
crime_pd_encoded = crime_pd.drop(['MCI_CATEGORY'], axis=1) # Drop the original MCI_CATEGORY column
crime_pd_encoded

,EVENT_UNIQUE_ID,NEIGHBOURHOOD_158,HOOD_158,LAT_WGS84,LONG_WGS84,PREMISES_TYPE,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_HOUR,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
0,GO-20213605,Wychwood (94),94,43.681769,-79.425762,Outside,1609477200000,2021,1,1,16,0,1,0,0,0
1,GO-20213400,NSA,0,0.000000,0.000000,Commercial,1609477200000,2021,1,1,4,0,1,0,0,0
2,GO-20211123,Yorkdale-Glen Park (31),31,43.721013,-79.460110,Other,1609477200000,2021,1,1,4,0,1,0,0,0
3,GO-2021445,Yonge-Doris (151),151,43.778743,-79.415293,Other,1609477200000,2021,1,1,1,0,1,0,0,0
4,GO-20213400,NSA,0,0.000000,0.000000,Commercial,1609477200000,2021,1,1,4,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147546,GO-20241427047,Cabbagetown-South St.James Town (71),71,43.663195,-79.373043,Apartment,1719637200000,2024,6,29,23,1,0,0,0,0
147547,GO-20241427869,York University Heights (27),27,43.759469,-79.464942,Commercial,1719723600000,2024,6,30,18,0,0,0,0,1
147548,GO-20241423116,Morningside Heights (144),144,43.837237,-79.248477,Outside,1719637200000,2024,6,29,21,1,0,0,0,0
147549,GO-20241426669,Mimico-Queensway (160),160,43.616490,-79.521053,Outside,1718859600000,2024,6,20,13,0,0,0,0,1


In [15]:
# Group the values by EVENT_UNIQUE_ID and get the first value for all the columns, except for Assault, Auto Theft, Break and Enter, Robbery, Theft Over
crime_first_group = crime_pd_encoded.groupby('EVENT_UNIQUE_ID')[["NEIGHBOURHOOD_158", "HOOD_158", "LAT_WGS84", "LONG_WGS84", "PREMISES_TYPE", "OCC_DATE", "OCC_YEAR", "OCC_MONTH", "OCC_DAY", "OCC_HOUR"]].first()
crime_first_group

,NEIGHBOURHOOD_158,HOOD_158,LAT_WGS84,LONG_WGS84,PREMISES_TYPE,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_HOUR
EVENT_UNIQUE_ID,,,,,,,,,,
GO-20211000033,West Queen West (162),162,43.646286,-79.408568,Commercial,1622264400000,2021,5,29,21
GO-2021100004,Morningside Heights (144),144,43.807252,-79.162903,Outside,1610773200000,2021,1,16,17
GO-20211000054,Moss Park (73),73,43.657067,-79.374531,Apartment,1622264400000,2021,5,29,22
GO-20211000193,Fort York-Liberty Village (163),163,43.636618,-79.399704,Apartment,1622264400000,2021,5,29,23
GO-20211000248,Eglinton East (138),138,43.737099,-79.246230,Outside,1622264400000,2021,5,29,21
...,...,...,...,...,...,...,...,...,...,...
GO-20249997,Junction-Wallace Emerson (171),171,43.668917,-79.442637,Outside,1704085200000,2024,1,1,18
GO-202499972,Edenbridge-Humber Valley (9),9,43.672705,-79.522472,House,1705208400000,2024,1,14,3
GO-2024999786,Flemingdon Park (44),44,43.718727,-79.334948,Apartment,1714539600000,2024,5,1,0


In [16]:
# Group the MCI_CATEGORY values by EVENT_UNIQUE_ID and get the maximum value.
crime_max_group = crime_pd_encoded.groupby('EVENT_UNIQUE_ID')[["Assault", "Auto Theft", "Break and Enter", "Robbery", "Theft Over"]].max()
crime_max_group

,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
EVENT_UNIQUE_ID,,,,,
GO-20211000033,0,0,1,0,0
GO-2021100004,0,1,0,0,0
GO-20211000054,1,0,0,0,0
GO-20211000193,1,0,0,0,0
GO-20211000248,1,0,0,0,0
...,...,...,...,...,...
GO-20249997,0,1,0,0,0
GO-202499972,0,1,0,0,0
GO-2024999786,1,0,0,0,0


In [17]:
# Concatenate the two groups together into one Dataframe and reset the index so that EVENT_UNIQUE_ID is not the index.
crime_groups_joined = pd.concat([crime_first_group, crime_max_group], axis=1)
crime_groups_joined = crime_groups_joined.reset_index()
crime_groups_joined

,EVENT_UNIQUE_ID,NEIGHBOURHOOD_158,HOOD_158,LAT_WGS84,LONG_WGS84,PREMISES_TYPE,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_HOUR,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
0,GO-20211000033,West Queen West (162),162,43.646286,-79.408568,Commercial,1622264400000,2021,5,29,21,0,0,1,0,0
1,GO-2021100004,Morningside Heights (144),144,43.807252,-79.162903,Outside,1610773200000,2021,1,16,17,0,1,0,0,0
2,GO-20211000054,Moss Park (73),73,43.657067,-79.374531,Apartment,1622264400000,2021,5,29,22,1,0,0,0,0
3,GO-20211000193,Fort York-Liberty Village (163),163,43.636618,-79.399704,Apartment,1622264400000,2021,5,29,23,1,0,0,0,0
4,GO-20211000248,Eglinton East (138),138,43.737099,-79.246230,Outside,1622264400000,2021,5,29,21,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129212,GO-20249997,Junction-Wallace Emerson (171),171,43.668917,-79.442637,Outside,1704085200000,2024,1,1,18,0,1,0,0,0
129213,GO-202499972,Edenbridge-Humber Valley (9),9,43.672705,-79.522472,House,1705208400000,2024,1,14,3,0,1,0,0,0
129214,GO-2024999786,Flemingdon Park (44),44,43.718727,-79.334948,Apartment,1714539600000,2024,5,1,0,1,0,0,0,0
129215,GO-2024999795,Oakridge (121),121,43.691225,-79.288346,Commercial,1715230800000,2024,5,9,13,1,0,0,0,0


---

## **Creating the Model**

## Testing Different Models

In [18]:
# Create a Total_Count column in the crime_groups_joined dataframe
crime_groups_joined['Total_Count'] = crime_groups_joined.iloc[:, 11:].sum(axis=1)
crime_groups_joined

,EVENT_UNIQUE_ID,NEIGHBOURHOOD_158,HOOD_158,LAT_WGS84,LONG_WGS84,PREMISES_TYPE,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_HOUR,Assault,Auto Theft,Break and Enter,Robbery,Theft Over,Total_Count
0,GO-20211000033,West Queen West (162),162,43.646286,-79.408568,Commercial,1622264400000,2021,5,29,21,0,0,1,0,0,1
1,GO-2021100004,Morningside Heights (144),144,43.807252,-79.162903,Outside,1610773200000,2021,1,16,17,0,1,0,0,0,1
2,GO-20211000054,Moss Park (73),73,43.657067,-79.374531,Apartment,1622264400000,2021,5,29,22,1,0,0,0,0,1
3,GO-20211000193,Fort York-Liberty Village (163),163,43.636618,-79.399704,Apartment,1622264400000,2021,5,29,23,1,0,0,0,0,1
4,GO-20211000248,Eglinton East (138),138,43.737099,-79.246230,Outside,1622264400000,2021,5,29,21,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129212,GO-20249997,Junction-Wallace Emerson (171),171,43.668917,-79.442637,Outside,1704085200000,2024,1,1,18,0,1,0,0,0,1
129213,GO-202499972,Edenbridge-Humber Valley (9),9,43.672705,-79.522472,House,1705208400000,2024,1,14,3,0,1,0,0,0,1
129214,GO-2024999786,Flemingdon Park (44),44,43.718727,-79.334948,Apartment,1714539600000,2024,5,1,0,1,0,0,0,0,1
129215,GO-2024999795,Oakridge (121),121,43.691225,-79.288346,Commercial,1715230800000,2024,5,9,13,1,0,0,0,0,1


In [19]:
# Group the crime_groups_joined dataframe by neighbourhood, occ_year, occ_month,
# get the count of the rows aggregated to each group, and place those values in the Total_Count column.
crime_totals_by_month_year_hood = crime_groups_joined.groupby(['HOOD_158', 'OCC_YEAR', 'OCC_MONTH'])[['Total_Count']].count()
crime_totals_by_month_year_hood = crime_totals_by_month_year_hood.reset_index()
crime_totals_by_month_year_hood

,HOOD_158,OCC_YEAR,OCC_MONTH,Total_Count
0,0,2021,1,44
1,0,2021,2,34
2,0,2021,3,45
3,0,2021,4,26
4,0,2021,5,37
...,...,...,...,...
6672,174,2024,2,15
6673,174,2024,3,7
6674,174,2024,4,17
6675,174,2024,5,12


In [20]:
# Group the crime_groups_joined dataframe by neighbourhood, occ_year, occ_month and get the sum of assaults, auto theft, break and enter, robbery, and theft over.
crime_types_by_month_year_hood = crime_groups_joined.groupby(['HOOD_158', 'OCC_YEAR', 'OCC_MONTH'])[["Assault", "Auto Theft", "Break and Enter", "Robbery", "Theft Over"]].sum()
crime_types_by_month_year_hood = crime_types_by_month_year_hood.reset_index()
# Concatenate crime_totals_by_month_year_hood and crime_types_by_month_year_hood intot a single dataframe
crime_by_month_year_hood = pd.concat([crime_types_by_month_year_hood, crime_totals_by_month_year_hood['Total_Count']], axis=1)
crime_by_month_year_hood.sort_values(by=['Total_Count'])

,HOOD_158,OCC_YEAR,OCC_MONTH,Assault,Auto Theft,Break and Enter,Robbery,Theft Over,Total_Count
1895,49,2021,6,1,0,1,0,0,1
2229,58,2021,4,0,0,1,0,0,1
1909,49,2022,8,0,1,0,0,0,1
5208,140,2021,2,1,0,0,0,0,1
1890,49,2021,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
77,1,2023,12,30,45,28,4,13,118
68,1,2023,3,18,83,12,3,3,119
65,1,2022,12,24,67,15,4,10,120
71,1,2023,6,24,77,22,4,5,131


In [21]:
# Show all the rows where HOOD_158 has a value of 0
crime_by_month_year_hood.loc[crime_by_month_year_hood['HOOD_158'] == 0, :]

,HOOD_158,OCC_YEAR,OCC_MONTH,Assault,Auto Theft,Break and Enter,Robbery,Theft Over,Total_Count
0,0,2021,1,23,2,10,9,1,44
1,0,2021,2,24,1,8,1,0,34
2,0,2021,3,27,7,5,5,3,45
3,0,2021,4,16,1,3,3,3,26
4,0,2021,5,30,3,3,1,1,37
5,0,2021,6,22,3,3,2,3,32
6,0,2021,7,29,2,3,6,2,42
7,0,2021,8,48,6,2,7,3,66
8,0,2021,9,27,8,6,3,3,47
9,0,2021,10,39,9,15,2,2,65


In [22]:
# Filter out the rows where the column HOOD_158 has a value of 0
crime_by_month_year_hood = crime_by_month_year_hood.loc[crime_by_month_year_hood['HOOD_158'] != 0, :]
crime_by_month_year_hood = crime_by_month_year_hood.reset_index(drop=True)
crime_by_month_year_hood

,HOOD_158,OCC_YEAR,OCC_MONTH,Assault,Auto Theft,Break and Enter,Robbery,Theft Over,Total_Count
0,1,2021,1,18,35,7,1,3,62
1,1,2021,2,17,17,5,1,3,43
2,1,2021,3,15,20,8,6,6,54
3,1,2021,4,11,31,4,2,4,52
4,1,2021,5,18,26,9,5,4,62
...,...,...,...,...,...,...,...,...,...
6630,174,2024,2,9,0,5,1,1,15
6631,174,2024,3,6,1,0,0,0,7
6632,174,2024,4,12,2,2,0,1,17
6633,174,2024,5,8,1,2,0,1,12


In [23]:
# Have the training data be all the data in crime_by_month_year_hood for the years 2021 to 2023
train_data = crime_by_month_year_hood.loc[crime_by_month_year_hood['OCC_YEAR'].isin([2021, 2022, 2023]), :]

# Have the testing data be all the data in crime_by_month_year_hood for the year 2024
test_data = crime_by_month_year_hood.loc[crime_by_month_year_hood['OCC_YEAR'] == 2024, :]

In [24]:
# Split the training data into the features and targets
X_train = train_data[['HOOD_158', 'OCC_YEAR', 'OCC_MONTH']]  # Features

X_test = test_data[['HOOD_158', 'OCC_YEAR', 'OCC_MONTH']]  # Features

y_train = train_data[['Total_Count']]  # Target

y_test = test_data[['Total_Count']]  # Target

In [25]:
# Initialize the Random Forest Regressor
r_model = RandomForestRegressor()

# Train the model on the training data
r_model.fit(X_train, y_train)

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [26]:
# Initialize the Histogram-Based Gradient Boosting Regressor and wrap it with MultiOutputRegressor
h_model = HistGradientBoostingRegressor()

# Train the model on the training data
h_model.fit(X_train, y_train)

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


HistGradientBoostingRegressor()

In [27]:
# Make predictions on the test set with the Random Forest Regressor
y_r_pred = h_model.predict(X_test)

# Make predictions on the test set with the Histogram-Based Gradient Boosting Regressor
y_h_pred = h_model.predict(X_test)

# Print the predicted crime counts for the test set for both models
print("Random Forest Predicted crime counts for (test set):", y_r_pred)
print("Histogram-Based Gradient Boosting Predicted crime counts for (test set):", y_h_pred)

Random Forest Predicted crime counts for (test set): [108.43872484 105.83106772 107.92372245 107.92372245 109.46283806
 109.46283806  40.51803705  37.91037993  41.1750202   41.1750202
  43.02688816  42.94401373  14.66910624  12.06144912  13.9615379
  13.9615379   13.13274684  13.04987242  14.66910624  12.06144912
  13.9615379   13.9615379   13.13274684  13.04987242  13.93657804
  11.32892092  13.22900969  13.22900969  12.29836593  12.21549151
  20.56521531  17.95755819  20.67916558  20.67916558  23.06526215
  23.06206052  20.03776149  17.43010437  20.15171176  20.15171176
  20.513163    20.37776691  10.7828557    8.75462203   9.74276584
   9.74276584  10.70315119  10.56775511  14.21406703  12.18583336
  13.82981035  13.82981035  15.23518015  15.44006918  14.21406703
  12.18583336  13.82981035  13.82981035  15.23518015  15.44006918
  15.10933576  13.08110208  14.98248568  14.98248568  16.8132207
  17.01810973  11.82309968   9.79486601  11.2739086   11.2739086
  11.40837179  11.27297571 

In [28]:
# Calculate the Mean Squared Error (MSE), Mean Absolute Error (MAE), and R-squared (R^2) for Random Forests.
r_mse = mean_squared_error(y_test, y_r_pred)
r_r2 = r2_score(y_test, y_r_pred)
r_mae = mean_absolute_error(y_test, y_r_pred)

print(f"Mean Squared Error: {r_mse}")
print(f"R-squared: {r_r2}")
print(f"mean absolute error: {r_mae}")

Mean Squared Error: 42.3805269624916
R-squared: 0.8042618644469353
mean absolute error: 5.0338833270972545


In [29]:
# Calculate the Mean Squared Error (MSE), Mean Absolute Error (MAE), and R-squared (R^2) for Histogram-Based Gradient Boosting.
h_mse = mean_squared_error(y_test, y_h_pred)
h_r2 = r2_score(y_test, y_h_pred)
h_mae = mean_absolute_error(y_test, y_h_pred)

print(f"Mean Squared Error: {h_mse}")
print(f"R-squared: {h_r2}")
print(f"mean absolute error: {h_mae}")

Mean Squared Error: 42.3805269624916
R-squared: 0.8042618644469353
mean absolute error: 5.0338833270972545


## Optimizing the models

In [30]:
# Create scorers for R-squared and MAE
r2_scorer = make_scorer(r2_score)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)  # Minimize MAE

In [31]:
# Define the hyperparameter search space for Histogram-Based Gradient Boosting Regressor
param_dist_hist = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'l2_regularization': [0.0, 0.1, 0.2]
}

# Create the Histogram-Based Gradient Boosting Regressor and wrap it with MultiOutputRegressor
hgb_model = HistGradientBoostingRegressor(random_state=1)

# Create RandomizedSearchCV object with multiple scoring metrics
random_search_hist = RandomizedSearchCV(
    estimator=hgb_model,
    param_distributions=param_dist_hist,
    n_iter=10,
    scoring={'r2': make_scorer(r2_score), 'mae': make_scorer(mean_absolute_error)},  # Multiple scorers
    refit='r2',  # Refit on R-squared
    cv=5,
    random_state=1,
    n_jobs=-1,
    return_train_score=True  # Optional: to get train scores for analysis
)

# Perform the hyperparameter search
random_search_hist.fit(X_train, y_train)

# Access results for both metrics
print(f"Best R-squared: {random_search_hist.best_score_}")  # Based on 'refit'
print(f"Best MAE: {-random_search_hist.cv_results_['mean_test_mae'][random_search_hist.best_index_]}")  # Negate MAE

# Get the best model
best_hgb_model = random_search_hist.best_estimator_

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

Best R-squared: -0.07234327285109235
Best MAE: -10.225096661187804


In [32]:
# Define the hyperparameter search space for Random Forest Regressor

# param_dist_reg = {
#     'n_estimators': [500, 700, 1000],          # Increase the number of trees
#     'max_depth': [15, 20, 25, None],           # Allow deeper trees with finer steps
#     'min_samples_split': [2, 3, 4, 5],         # Allow more splits
#     'min_samples_leaf': [1, 2, 3],             # Smaller leaf sizes
#     'max_features': [0.3, 0.5, 'sqrt', 'log2'],# Additional options for max features
#     'bootstrap': [True, False]                 # Both bootstrap sampling options
# }

# # Create the model for Random Forest Regressor
# rf_model = RandomForestRegressor(random_state=1)

# # Create RandomizedSearchCV object (similar to HGB)
# random_search_reg = RandomizedSearchCV(
#     estimator=rf_model,
#     param_distributions=param_dist_reg,
#     n_iter=25,
#     scoring={'r2': r2_scorer, 'mae': mae_scorer},  # Multiple scorers
#     refit='r2',  # Refit on R-squared
#     cv=10,
#     random_state=1,
#     n_jobs=-1
# )

# # Perform the hyperparameter search
# random_search_reg.fit(X_train, y_train)

# # Get the best hyperparameters and model (similar to HGB)
# best_params = random_search_reg.best_params_
# print(f"Best Hyperparameters: {best_params}")
# best_rf_model = random_search_reg.best_estimator_

param_dist_reg = {
    'n_estimators': [1000, 1200, 1500],          # Increase the number of trees for better averaging
    'max_depth': [20, 30, 40, None],             # Allow deeper trees for complex relationships
    'min_samples_split': [2, 5, 10],             # Increase min_samples_split to reduce overfitting
    'min_samples_leaf': [1, 2, 4],               # Experiment with smaller leaf sizes for finer granularity
    'max_features': [0.7, 0.8, 'sqrt', 'log2'],  # Use a larger fraction of features to improve learning
    'bootstrap': [True]                          # Use bootstrap sampling (often performs better for Random Forest)
}

# Create the model for Random Forest Regressor
rf_model = RandomForestRegressor(random_state=1)

# Create RandomizedSearchCV object (similar to HGB)
random_search_reg = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_dist_reg,
    n_iter=30,                                  # Increase number of iterations to explore more configurations
    scoring={'r2': make_scorer(r2_score), 'mae': make_scorer(mean_absolute_error)},  # Multiple scorers
    refit='r2',  # Refit on R-squared
    cv=10,       # Increase number of folds for cross-validation to reduce variance in scoring
    random_state=1,
    n_jobs=-1
)

# Perform the hyperparameter search
random_search_reg.fit(X_train, y_train)

# Get the best hyperparameters and model
best_params = random_search_reg.best_params_
print(f"Best Hyperparameters: {best_params}")
best_rf_model = random_search_reg.best_estimator_

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/dev/lib/python3.12/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

Best Hyperparameters: {'n_estimators': 1500, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': True}


In [38]:
# Calculate the Mean Squared Error (MSE), Mean Absolute Error (MAE), and R-squared (R^2) for Random Forests.
r_mse = mean_squared_error(y_test, y_r_pred)
r_r2 = r2_score(y_test, y_r_pred)
r_mae = mean_absolute_error(y_test, y_r_pred)

print(f"Mean Squared Error: {r_mse}")
print(f"R-squared: {r_r2}")
print(f"mean absolute error: {r_mae}")

Mean Squared Error: 97.3061991470195
R-squared: 0.5505828887958442
mean absolute error: 7.50735529249953


In [39]:
# Make predictions on the test set with the Random Forest Regressor
y_r_pred = best_rf_model.predict(X_test)

# Make predictions on the test set with the Histogram-Based Gradient Boosting Regressor
y_h_pred = best_hgb_model.predict(X_test)

# Print the predicted crime counts for the test set for both models
print("Random Forest Predicted crime counts for (test set):", y_r_pred)
print("Histogram-Based Gradient Boosting Predicted crime counts for (test set):", y_h_pred)

Random Forest Predicted crime counts for (test set): [103.06413181 100.07466035 103.82579363 103.08904527 105.162904
 106.63668594  31.50358686  29.28182509  28.62547494  33.75177497
  35.87311729  38.04717918  18.32358691  14.55412201  18.41041025
  18.27286308  17.0526802   16.0575035   16.93762334  12.81028162
  17.91898219  15.19251028  14.6744839   13.02159488  16.02345141
  12.66767732  17.84981687  14.72238957  14.4368828   12.72928139
  16.58128563  13.35496247  19.21447257  15.78386566  17.95647357
  16.89863432  16.76424719  13.43463188  19.35028262  14.83252406
  16.47088709  16.46694381  15.53404583  11.10486258  14.19110994
  12.89056184  14.57216144  14.86748243  15.86538958  11.4427126
  13.37523784  13.32131949  15.6497985   16.15220639  16.01179492
  11.49370138  13.55304011  13.32443528  15.64026269  16.10904928
  16.14833506  11.51069107  13.63889928  13.01668042  15.40393649
  15.96286502  15.64680535  11.25777543  13.31879975  12.39224517
  14.41286095  13.96716154

In [40]:
# Calculate the Mean Squared Error (MSE), Mean Absolute Error (MAE), and R-squared (R^2) for Random Forests.
r_mse = mean_squared_error(y_test, y_r_pred)
r_r2 = r2_score(y_test, y_r_pred)
r_mae = mean_absolute_error(y_test, y_r_pred)

print(f"Mean Squared Error: {r_mse}")
print(f"R-squared: {r_r2}")
print(f"mean absolute error: {r_mae}")

Mean Squared Error: 97.3061991470195
R-squared: 0.5505828887958442
mean absolute error: 7.50735529249953


In [41]:
# Calculate the Mean Squared Error (MSE), Mean Absolute Error (MAE), and R-squared (R^2) for Histogram-Based Gradient Boosting.
h_mse = mean_squared_error(y_test, y_h_pred)
h_r2 = r2_score(y_test, y_h_pred)
h_mae = mean_absolute_error(y_test, y_h_pred)

print(f"Mean Squared Error: {h_mse}")
print(f"R-squared: {h_r2}")
print(f"mean absolute error: {h_mae}")

Mean Squared Error: 54.89960669312779
R-squared: 0.7464414100792103
mean absolute error: 5.745211113851994
